# Исследование надежности заемщиков


## Изучим общую информацию о данных

In [28]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [29]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Задание 4. Выведем количество пропущенных значений для каждого столбца.

In [31]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [32]:
#На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце  медианным значением по каждому типу из столбца income_type
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [33]:
#замените все отрицательные значения положительными с помощью метода abs()
data['days_employed'] = data['days_employed'].abs()

**Задание 7. Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.**

In [34]:
#Для каждого типа занятости выведите медианное значение трудового стажа days_employed в днях
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получаются аномально большие значения. Оставим так как есть.

In [35]:
#Выведем перечень уникальных значений столбца `children`
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [36]:
#удалим аномальные значения
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [37]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков.

In [38]:
#Заполним пропуски в столбце days_employed медианными значениями для каждого типа занятости income_type
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [39]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [40]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [41]:
#Обработаем неявнфе дубликаты, приведем к нижнему регистру.
data['education'] = data['education'].str.lower()

In [42]:
#выведем кол-во строк дубликатов
data.duplicated().sum()

71

In [43]:
# удалим дубликаты
data = data.drop_duplicates()

### Категоризация данных

In [44]:
# категоризируем данные
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [45]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [46]:
# перечень уникальных целей взятия кредита
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [47]:
# создадим новый столбец с категориями через функцию.
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [48]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследование  данных .

#### Гипотеза:
Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [49]:
count_children = data.groupby('children')['debt'].agg(['sum','count']) # сгруппируем данные по столбцу 'children',посчитаем общее кол- во кредитов по группам и сумму просроченных кредитов.
result_children = count_children['sum']/count_children['count']  # посчитаем долю просроченных кредитов.
print(result_children*100)
print(count_children)


children
0    7.543822
1    9.234609
2    9.454191
3    8.181818
4    9.756098
5    0.000000
dtype: float64
           sum  count
children             
0         1063  14091
1          444   4808
2          194   2052
3           27    330
4            4     41
5            0      9


**Вывод:** 
1. Большая часть заемщиков - это люди без детей. На их долю приходится 14091 кредитов. Они же чаще всего возращают кредиты в срок.
2. Второе и третье место по количеству кредитов приходится на заемщиков с 1 и 2-мя детьми. Среди них высокий процент просрочек.
3. Заемщики у которых 3-е детей реже обращаются за кредитами и аккуратнее относятся к возвратам по сравнению с заемщиками с одним и двумя детьми.
4. Заемщики с 4-мя детьми имеют небольшой процент кредитования и наихудший показатель по возвратам, процент просрочки = 9,8%.
5. Меньше всего кредитов приходится на долю заемщиков с 5-ю детьми. Кол-во просрочек у них равно 0.

#### Гипотеза:
Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [50]:
family_status_count_sum = data.groupby('family_status')['debt'].agg(['sum','count'])  # сгруппируем данные по столбцу 'family_status',
                                                                                     # посчитаем общее кол- во кредитов по группам и сумму просроченных кредитов.
result_family_status = family_status_count_sum['sum']/family_status_count_sum['count'] # посчитаем долю просроченных кредитов.
print(family_status_count_sum)
print(result_family_status*100)


                       sum  count
family_status                    
Не женат / не замужем  273   2796
в разводе               84   1189
вдовец / вдова          63    951
гражданский брак       385   4134
женат / замужем        927  12261
family_status
Не женат / не замужем    9.763948
в разводе                7.064760
вдовец / вдова           6.624606
гражданский брак         9.313014
женат / замужем          7.560558
dtype: float64


**Вывод:** 
1. Большая для кредитов приходится на семейные пары. У этой категории заемщиков низкий процент просрочек по кредиту(7.6%). 
2. Второе место по кредитованию у пар в гражданском браке. У этой категории заемщиков высокий процент просрочек(9.3%)
3. Заемщики в статусе "свободен(на)" занимают 3-е место по числу кредитов и первое место по просрочкам(9.8%)
4. Заемщики в статусе "разведен(на)" аккуратно выплачивают кредит.
4. На категоря вдовец(ва) приходится меньше всего кредитов. Они самые аккуратные заемщики(6.6%).
Большая часть кредиторов - семеные пары. Займы выплачивают аккуратно. 
Во вторую группу можно объединить пары, которые проживают в гражданском браке или заемщики в статусе "свободен(на)". У этих групп самый высокий процент по просрочкам. Заемщики в разводе составляют небольшую группу, которая имеет небольшой процент по просрочкам. Самая маленькая группа заемщиков с лучшими показателями - это вдовы(цы). 

#### Гипотеза:
Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [51]:
income_by_category_count_sum = data.groupby('total_income_category')['debt'].agg(['sum','count'])  #сгруппируем данные по столбцу 'total_income_category',посчитаем общее кол- во кредитов по группам и сумму просроченных кредитов.
result_income_by_category = income_by_category_count_sum['sum']/income_by_category_count_sum['count']  # посчитаем долю просроченных кредитов.
print(income_by_category_count_sum)
print(result_income_by_category*100)


                        sum  count
total_income_category             
A                         2     25
B                       354   5014
C                      1353  15921
D                        21    349
E                         2     22
total_income_category
A    8.000000
B    7.060231
C    8.498210
D    6.017192
E    9.090909
dtype: float64


**Вывод:** 
1. В результате анализа можно выделить 2 небольшие группы. Это заемщики категории 'A' и "Е", люди с максимально низким и максимально высоким доходом имеют высокий процент просрочек по кредиту. для того чтобы ответить на поставленный вопрос по данной категории нужно больше данных.
2. Самая крупная категория заемщиков - это люди с доходом от 50 000 до 200 000 рубл. Их также нельзя отнести к аккуратным плательщикам.
3. Заемщики категории "В"  входят во вторую по размерам группу. Можно сказать, что это люди с высоким доходом, которые исправно возращают займ.
4. Небольшая группа заемщиков в категории "D". Это люди с невысоким  доходом и самыми лучшими показателями по выплатам. 

#### Гипотеза:
Как разные цели кредита влияют на его возврат в срок?

In [52]:
purpose_category_count_sum = data.groupby('purpose_category')['debt'].agg(['sum','count'])  #сгруппируем данные по столбцу 'purpose_category',посчитаем общее кол- во кредитов по группам и сумму просроченных кредитов.
result_purpose_category = purpose_category_count_sum['sum']/purpose_category_count_sum['count']# посчитаем долю просроченных кредитов.
print(purpose_category_count_sum)
print(result_purpose_category*100)


                          sum  count
purpose_category                    
операции с автомобилем    400   4279
операции с недвижимостью  780  10751
получение образования     369   3988
проведение свадьбы        183   2313
purpose_category
операции с автомобилем      9.347978
операции с недвижимостью    7.255139
получение образования       9.252758
проведение свадьбы          7.911803
dtype: float64


**Вывод:** 
1. Большая доля кредитов приходится на операции с недвижимостью. По этой категории самые высокие показатели своевременных выплат.
2. Вторая по численности доля кредитов приходится на покупку автомобиля. У этой категории наихудший показатель по просрочке выплат.
3. Кредиты на получение образования также имеют высокий процент просрочек.
4. Высокий показатель по своевременным выплатам у заемщиков, которые берут кредит на свадьбу.

In [53]:
print( data.groupby('gender')['debt'].sum() * 100 / data.groupby('gender')['debt'].count())
print( data.groupby('gender')['debt'].count())

gender
F       7.018166
M      10.265267
XNA     0.000000
Name: debt, dtype: float64
gender
F      14092
M       7238
XNA        1
Name: debt, dtype: int64


Вывод: Большая часть кредиторов - женщины, мужчин на 50% меньше. С чем это связано сложно сказать. Может быть, потому что женщин в России больше чем мужчин. Может быть, потому что мужчинам не одобряют кредит повторно и они оформляют их на жен. Невозможно дать точный ответ на этот вопрос,т.к. недостаточно данных. Но факт остается фактом -заемщики женщины более ответственно относятся к выплате кредитов по сравнению с заемщиками мужчинами. 

###  Общий вывод.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
В процессе анализа данных были выявлены пропуски в столбцах "days_employed " и "days_employed ". Пропуски заполнили значениями по медиане. Выявленные дубликаты обработаны. Данные по целям кредита и доходам клиента разбиты по категориям для удобства анализа.
Проанализировав данные можно сделать следующие выводы:
1. Самые надежные заемщики - это семьи без детей, с высоким доходом, которые планируют покупку недвижимости.
2. Самые ненадежные - это свободные заемщики( либо состоящие в гражданском браке) с низким доходом, которые планируют покупку автомобиля.
3. Большая часть кредитов была одобрена женщинам, они же аккуратнее относятся к выплатам.

